In [79]:
import os
import json
import requests
from collections import OrderedDict 
from dotenv import load_dotenv
import openai
import logging
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import  SimpleSpanProcessor, ConsoleSpanExporter
from azure.core.settings import settings 
from IPython.display import Markdown, display
from azure.monitor.opentelemetry import configure_azure_monitor
from azure.monitor.opentelemetry.exporter import AzureMonitorTraceExporter

load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
exporter = AzureMonitorTraceExporter.from_connection_string(
    os.environ["APPLICATION_INSIGHTS_CONNECTION_STRING"]
)

tracer_provider = TracerProvider()
# Setup tracing to console
# Requires opentelemetry-sdk
span_exporter = ConsoleSpanExporter()
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter))
trace.set_tracer_provider(tracer_provider)

# [START trace_function]
from opentelemetry.trace import get_tracer

tracer = get_tracer(__name__)

settings.tracing_implementation = "opentelemetry"

os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'
# Enable Azure Monitor tracing
application_insights_connection_string = os.getenv("APPLICATION_INSIGHTS_CONNECTION_STRING")

if not application_insights_connection_string:
    print("Application Insights was not enabled for this project.")
    print("Enable it via the 'Tracing' tab in your Azure AI Foundry project page.")
    exit()
    
configure_azure_monitor(connection_string=application_insights_connection_string)


openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-03-15-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
bing_search_url = os.getenv("BING_SEARCH_API_BASE")
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_KEY")
search_index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
search_api_version = os.getenv("AZURE_SEARCH_API_VERSION")


In [80]:
# messages=[
#     {"role": "system", "content": "You are an AI assistant that is a travel planning expert especially with National Parks."},
#     {"role": "user", "content": "List top 5 important things to develop a modern AI Application or Agent."},
# ]


# res = openai.ChatCompletion.create(
#     engine=os.getenv("ENGINE"),
#     messages = messages,
#     temperature=0.01,
#     max_tokens=800,
#     top_p=0.01
#     )

# display(Markdown(res.choices[0].message.content))


In [81]:
@tracer.start_as_current_span(name="pci_dss_v4_function_calling")
def pci_dss_v4(query)->str:
    """
    This function uses the OpenAI API to generate a response to a query about PCI DSS v4.
    """
    headers = {'Content-Type': 'application/json','api-key': search_key}
    params = {'api-version': search_api_version}
    k = 5
    reranker_threshold =1

    search_payload = {
        "search": query,
        "queryType": "semantic",
        "semanticConfiguration": "azureml-default",
        "count": "true",
        "speller": "lexicon",
        "queryLanguage": "en-us",
        "captions": "extractive",
        "answers": "extractive",
        "top": k
    }

    search_payload["select"]= "id, title, content, filepath, url"
    

    resp = requests.post(search_endpoint + "/indexes/" + search_index_name + "/docs/search",
                        data=json.dumps(search_payload), headers=headers, params=params,)
    
    search_results = resp.json()

    content = dict()
    ordered_content = OrderedDict()

    for result in search_results['value']:
        if result['@search.rerankerScore'] > reranker_threshold: # Show results that are at least N% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'], 
                                    "filepath": result['filepath'], 
                                    "url": result['url'],
                                    "caption": result['@search.captions'][0]['text']
                                }
            content[result['id']]["content"]= result['content']
            content[result['id']]["score"]= result['@search.score'] # Uses the Hybrid RRF score
                
    # After results have been filtered, sort and add the top k to the ordered_content
        
    count = 0  # To keep track of the number of results added
    for id in sorted(content, key=lambda x: content[x]["score"], reverse=True):
        ordered_content[id] = content[id]
        count += 1
        if count >= k:  # Stop after adding k results
            break

    return json.dumps(ordered_content)

In [82]:
# openai function calling 
functions = [  
    {
        "name": "pci_dss_v4",
        "description": "Searches for PCI DSS v4 related information.",
        "parameters":{
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "PCI DSS related search query",
                }
            },
            "required": ["query"],
        }
    }
]

available_functions = {
    "pci_dss_v4": pci_dss_v4
}

In [83]:
@tracer.start_as_current_span(name="run_multiturn_conversation")
def run_multiturn_conversation(messages, functions, available_functions, deployment_name, verbose: bool=False):
    """
    Runs a multi-turn conversation with GPT4, where GPT4 may call functions
    :param messages: a list of messages in the conversation, where each message is a dict with keys "role" and "content"
    :param functions: a list of functions that GPT4 can call
    :param available_functions: a dict of function names to functions
    :param deployment_name: the name of the deployment to use


    """
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_name,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0
    )

    if verbose:
        print(response)

    # Step 2: check if GPT wanted to call a function
    while response["choices"][0]["finish_reason"] == 'function_call':
        if verbose:
            print("== Response Type: function_call {} ==".format(response["choices"][0]["finish_reason"]))
        
        response_message = response["choices"][0]["message"]
        
        if verbose:
            print("Recommended Function call:")
            print(response_message.get("function_call"))
            print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]  
        
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(**function_args)
        
        if verbose:
            print(function_name, "Output of function call:", function_response)
            print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "function_call": {
                    "name": response_message["function_call"]["name"],
                    "arguments": response_message["function_call"]["arguments"],
                },
                "content": None
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        if verbose:
            print("== Messages in next request:")
            for message in messages:
                print(message)
            print("=============================")

        response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_name,
            function_call="auto",
            functions=functions,
            temperature=0
        )  # get a new response from GPT where it can see the function response

        if verbose:
            print(response)

    return response, messages


In [84]:
user_message = """
what should i check to make sure i'm complying with PCI DSS for database protection?
"""

In [85]:
system_message = """You are an assistant designed to help business analyist and data analyist answer questions.
You have access to PCI DSS v4 document. You should call pci_dss_v4 whenever a question requires information from PCI DSS v4 document.

In your answer, repeast the question and then call the appropriate function to get the answer.

## Safety

If you don't have funtion or skill to answer a question, say you don't know. 

## Response

If you used a Function Calling state that which function you used
Reponse format
[Funtion `function_name` called]

[Your ansers]

### Example 1
[Function `search_bing` called] 

The latest price of Bitcoin is approximately $38,808.75 USD

### Example 2
[Function `retrieve_sales_report` called]

Latest date in the sales report is from May 2016. 

"""

messages = [{"role": "system", "content": system_message},
            {"role": "user", "content": user_message}]


result, conversations = run_multiturn_conversation(messages, functions, available_functions, os.getenv("ENGINE"), verbose=False)

display(Markdown(result['choices'][0]['message']['content']))

To ensure compliance with PCI DSS for database protection, you should focus on the following key requirements:

1. **Protect Stored Account Data (Requirement 3)**:
   - **3.1**: Define and understand processes and mechanisms for protecting stored account data.
   - **3.2**: Minimize the storage of account data.
   - **3.3**: Do not store sensitive authentication data (SAD) after authorization.
   - **3.4**: Restrict access to displays of full PAN and the ability to copy cardholder data.
   - **3.5**: Secure the primary account number (PAN) wherever it is stored.
   - **3.6**: Secure cryptographic keys used to protect stored account data.
   - **3.7**: Implement key management processes and procedures covering all aspects of the key lifecycle when cryptography is used to protect stored account data.

2. **Use of Protection Methods**:
   - Utilize encryption, truncation, masking, and hashing to protect account data.
   - Ensure that encrypted account data is unreadable without the proper cryptographic keys.
   - Consider minimizing risk by not storing account data unless necessary, truncating cardholder data if full PAN is not needed, and avoiding sending unprotected PANs using end-user messaging technologies such as email and instant messaging.

3. **Non-Persistent Memory**:
   - If account data is present in non-persistent memory (e.g., RAM, volatile memory), encryption is not required. However, proper controls must be in place to ensure that memory maintains a non-persistent state and data is removed once the business purpose is complete.

4. **Key Management**:
   - Implement key management policies and procedures to include the generation of strong cryptographic keys, secure distribution of cryptographic keys, and protection of cryptographic keys against disclosure and misuse.

By adhering to these requirements, you can ensure that your database protection measures are in compliance with PCI DSS standards.

[Function `pci_dss_v4` called]